In [2]:
from __future__ import print_function

import sys
import numpy as np
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split

sys.path.append('..')
import tensorflow as tf
#from supplementary_code_direct_ranker.DirectRanker import directRanker
#from supplementary_code_direct_ranker.helpers import readData, nDCGScorer_cls, MAP_cls
import DirectRanker as drr
import helpers as hlps

Instructions for updating:
non-resource variables are not supported in the long term


# Read data

In [11]:
# x_test, y_test, q_test = hlps.readData(data_path="/Users/wery/Desktop/BAJiawenWang/dataset/MQ2007/Fold1/test.txt", 
#                                        binary=True, at=10, number_features=46, bin_cutoff=1.5, cut_zeros=True)
# x_train, y_train, q_train = hlps.readData(data_path="/Users/wery/Desktop/BAJiawenWang/dataset/MQ2007/Fold1/train.txt", 
#                                           binary=True, at=10, number_features=46, bin_cutoff=1.5, cut_zeros=True)

# x_test, y_test, q_test = hlps.readData(data_path="/Users/wery/Desktop/BAJiawenWang/dataset/MQ2008/Fold1/test.txt", 
#                                        binary=True, at=10, number_features=46, bin_cutoff=1.5, cut_zeros=True)
# x_train, y_train, q_train = hlps.readData(data_path="/Users/wery/Desktop/BAJiawenWang/dataset/MQ2008/Fold1/train.txt", 
#                                           binary=True, at=10, number_features=46, bin_cutoff=1.5, cut_zeros=True)

x_test, y_test, q_test = hlps.readData(data_path="/Users/wery/Desktop/BAJiawenWang/dataset/OHSUMED/Feature-min/Fold1/testset.txt", 
                                       binary=True, at=10, number_features=45, bin_cutoff=1.5, cut_zeros=True)
x_train, y_train, q_train = hlps.readData(data_path="/Users/wery/Desktop/BAJiawenWang/dataset/OHSUMED/Feature-min/Fold1/trainingset.txt", 
                                          binary=True, at=10, number_features=45, bin_cutoff=1.5, cut_zeros=True)

# x_test, y_test, q_test = hlps.readData(data_path="/Users/wery/Desktop/BAJiawenWang/dataset/TREC/TD2003/Fold1/testset.txt", 
#                                        binary=False, at=10, number_features=44, bin_cutoff=1.5, cut_zeros=True)
# x_train, y_train, q_train = hlps.readData(data_path="/Users/wery/Desktop/BAJiawenWang/dataset/TREC/TD2003/Fold1/trainingset.txt", 
#                                           binary=False, at=10, number_features=44, bin_cutoff=1.5, cut_zeros=True)

# x_test, y_test, q_test = hlps.readData(data_path="/Users/wery/Desktop/BAJiawenWang/dataset/TREC/TD2004/Fold1/testset.txt", 
#                                        binary=False, at=5, number_features=44, bin_cutoff=1.5, cut_zeros=True)
# x_train, y_train, q_train = hlps.readData(data_path="/Users/wery/Desktop/BAJiawenWang/dataset/TREC/TD2004/Fold1/trainingset.txt", 
#                                           binary=False, at=5, number_features=44, bin_cutoff=1.5, cut_zeros=True)

# x_test, y_test, q_test = hlps.readData(data_path="/Users/wery/Desktop/BAJiawenWang/dataset/MSLR-WEB10K/Fold1/test.txt", 
#                                        binary=True, at=10, number_features=136, bin_cutoff=1.5, cut_zeros=True)
# x_train, y_train, q_train = hlps.readData(data_path="/Users/wery/Desktop/BAJiawenWang/dataset/MSLR-WEB10K/Fold1/train.txt", 
#                                           binary=True, at=10, number_features=136, bin_cutoff=1.5, cut_zeros=True)

#only 2 min readData!!!

In [4]:
#len(x_train) #60??? Why not 63 OHSUMED???

#len(q_train) #shape (9219, ) correct # test:3383
#len(x_test) #20??? Why not 22 OHSUMED??? already different at 2nd qid86
y_train[0].ravel()

array([0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0])

In [5]:
x_train[0]

array([[ 0.84744973,  0.91581777,  0.55976752, ..., -0.16976018,
        -0.19343922, -0.15279566],
       [ 0.43178751,  0.44776221,  0.6712238 , ..., -0.15749394,
        -0.17838753, -0.12414577],
       [ 0.2177922 ,  0.22247301,  0.34741558, ..., -0.18609576,
        -0.20246483, -0.15652423],
       ...,
       [-0.01171172, -0.01171172, -0.02385716, ..., -0.55820607,
        -0.55181484, -0.57256782],
       [ 0.2177922 ,  0.22247301, -0.06743775, ..., -0.58739211,
        -0.55170028, -0.58452112],
       [-1.73311253, -1.73311253, -1.73311253, ..., -0.58739211,
        -0.55181484, -0.58452112]])

00: 138, correct, qid1  
01: 153 += 291, correct, qid2  
02: 235 += 526, correct, qid3  
03: 111 += 637, correct, qid4  
04: 56 += 693, correct, qid5  
05: 153 += 846, correct, qid6  
06: 54 += 900, correct, qid7  
07: 140 += 1040 ??? 942, qid8 all0  
08: 35 += 1075 ??? 1082 = 942+140, qid9   
09: 96 += 1171 ??? 1117 = 1082+35, qid10  
10: 84 += 1255 ??? 1213 = 1117+96, qid11  
11: 81 += 1336 ??? 1297 = 1213+84, qid12  
12: 133 += 1469 ??? 1378 = 1297+81, qid13  
13: 134 += 1603 ??? 1511 = 1378+133, qid14  
14: 215 += 1818 ??? 1645 = 1511+134, qid15  
15: 112 += 1930 ??? 1860 = 1645+215, qid16  
16: 155 += 2085 ??? 1972 = 1860+112, qid17  
17: 134 += 2219 ??? 2127 = 1972+155, qid18   
18: 134 += 2353 ??? 2261 = 2127+134, qid19  
19: 175 += 2528 ??? 2395 = 2261+134, qid20  
20: 205 += 2733 ??? 2570 = 2395+175, qid21  
21: 63 += 2796 ??? 2775 = 2570+205, qid22  
22: 41 += 2837 ??? 2838 = 2775+63, qid23  
23: 54 += 2891 ??? 2879 = 2838+41, qid24  
24: 83 += 2974 ??? 2933 = 2879+54, qid25  
25: 268 += 3242 ??? 3016 = 2933+83, qid26  
26: 174 += 3416 ??? 3284 = 3016+268, qid27  
27: 150 += 3566 ??? 3391 = 3284+174 ??? 3391, qid28  
28: 137 += 3703 ??? 3565, qid29  
29: 132 += 3835 ??? 3715, qid30  
30: 243 += 4078 ??? 3852, qid31  
31: 193 += 4271 ??? 3984, qid32  
32: 320 += 4591 ??? 4227, qid33  
33: 133 += 4724 ??? 4420, qid34  
34: 147 += 4871 ??? 4740, qid35  
35: 81 += 4952 ??? 4873, qid36  
36: 146 += 5098 ??? 5020, qid37  
37: 135 += 5233 ??? 5101, qid38  
38: 184 += 5417 ??? 5247, qid39  
39: 80 += 5497 ??? 5382, qid40  
40: 180 += 5677 ??? 5566, qid41  
41: 131 += 5808 ??? 5646, qid42  
42: 205 += 6013 ??? 5826, qid43  
43: 145 += 6158 ??? 5957, qid44  
44: 172 += 6330 ??? 6162, qid45  
45: 262 += 6592 ??? 6307, qid46  
46: 167 += 6759 ??? 6479, qid47  
47: 159 += 6918 ??? 6741, qid48  
48: 129 += 7047 ??? 6850, qid49  
49: 241 += 7288 ??? 7017, qid50  
50: 288 += 7576 ??? 7176, qid51  
51: 172 += 7748 ??? 7305, qid52  
52: 130 += 7878 ??? 7546, qid53  
53: 165 += 8043 ??? 7834, qid54  
54: 218 += 8216 ??? 8006, qid55   
55: 114 += 8375 ??? 8136, qid56  
56: 133 += 8508 ??? 8301, qid57  
57: 74 += 8582 ??? 8519, qid58  
58: 156 += 8738 ??? 8633, qid59  
59: 223 += 8961 ??? 8766, qid60  
   
??? 8840, qid61  
??? 8996, qid62  
??? 9219, qid63  

In [6]:
print(q_train) # [ 1  1  1 ... 63 63 63]

[ 1  1  1 ... 63 63 63]


# Define Ranker

In [7]:
def lambda_cost(nn, y0):
    return tf.reduce_mean(tf.log(1+tf.exp(nn))-nn)


# Load directRanker, train, and test
dr = drr.directRanker(
    feature_activation=tf.nn.tanh,
    ranking_activation=tf.nn.tanh,
    # max_steps=10000,
    # For debugging
    #cost=lambda_cost,
    max_steps=10000,
    print_step=500,
    start_batch_size=3,
    end_batch_size=5,
    start_qids=20,
    end_qids=100,
    feature_bias=True,
    hidden_layers=[100, 50, 5]
)

# Fit

In [8]:
dr.fit(x_train, y_train, ranking=True)

2023-03-05 01:11:56.809283: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-03-05 01:11:56.815591: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


step: 0, value: 1.6615132093429565, samples: 3, queries: 20
step: 500, value: 0.7046020030975342, samples: 3, queries: 21
step: 1000, value: 0.7999551296234131, samples: 3, queries: 23
step: 1500, value: 0.5364226698875427, samples: 3, queries: 25
step: 2000, value: 0.4375348389148712, samples: 3, queries: 27
step: 2500, value: 0.6053845286369324, samples: 3, queries: 29
step: 3000, value: 0.3740880489349365, samples: 3, queries: 32
step: 3500, value: 0.2799215018749237, samples: 3, queries: 35
step: 4000, value: 0.3313451111316681, samples: 3, queries: 38
step: 4500, value: 0.27198800444602966, samples: 3, queries: 41
step: 5000, value: 0.20802217721939087, samples: 3, queries: 44
step: 5500, value: 0.22796548902988434, samples: 3, queries: 48
step: 6000, value: 0.14735272526741028, samples: 4, queries: 52
step: 6500, value: 0.18075814843177795, samples: 4, queries: 56
step: 7000, value: 0.12611159682273865, samples: 4, queries: 61
step: 7500, value: 0.16948272287845612, samples: 4, q

# Evaluate

In [9]:
hlps.nDCGScorer_cls(dr, x_test, y_test, at=5)
hlps.MAP_cls(dr, x_test, y_test)

nDCG@5: 0.135 +- 0.1592
MAP: 0.1692 +- 0.0992


0.1692289673912754

In [10]:
# OHSUMED:
# nDCG@5: 0.2617 +- 0.3008
# MAP: 0.2484 +- 0.2183

# MQ2008
# nDCG@5: 0.4944 +- 0.3143
# MAP: 0.4663 +- 0.2943
# nDCG@10: 0.5661 +- 0.279
# MAP: 0.4663 +- 0.2943

# MSLR-WEB10K
# nDCG@10: 0.4109 +- 0.2404
# MAP: 0.3452 +- 0.178